# PyTorch Transfer Learning

Transfer learning is the usage of models that have been trained on general data to solve a more specific problem.  
PyTorch offers pre-trained models in `torchvision.models`, `torchtext.models`, `torchaudio.models`, and `torchrec.models`.  
Models can also be found on HuggingFace, `timm`, and Paperswithcode.

## Setup

In [1]:
import torch
import torchvision

In [2]:
print(torch.__version__)
print(torchvision.__version__)

2.1.2
0.16.2


In [3]:
import matplotlib.pyplot as plt

In [4]:
from torch import nn
from torchvision import transforms

In [5]:
try:
    from torchinfo import summary
except:
    print("[INFO] Could not find torchinfo, installing")
    !pip install -q torchinfo
    from torchinfo import summary

In [6]:
try:
    from going_modular.going_modular import data_setup, engine
except:
    print("[INFO] Could not find going_modular scripts, installing")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

## Get datas

In [10]:
import os
import zipfile
import requests

In [11]:
from pathlib import Path

In [13]:
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"
if image_path.is_dir():
    print(f"{image_path} exists")
else:
    print(f"Could not find {image_path}, creating")
    image_path.mkdir(parents=True, exist_ok=True)
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading data")
        f.write(request.content)
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unzipping data")
        zip_ref.extractall(image_path)
    os.remove(data_path / "pizza_steak_sushi.zip")

data/pizza_steak_sushi exists


In [14]:
train_dir = image_path / "train"
test_dir = image_path / "test"

## Create Datasets and DataLoaders

The data going into a pre-trained model is expected to fit certain standards, which is achieved by applying the relevant transforms.  
The data has to be in form of mini-batches of data of shape `3 x H x W`, where `H` and `W` each have to be at least `224`.  
Each value has to be in the range `[0, 1]`, with mean `[0.485, 0.456, 0.406]` and standard deviation `[0.229, 0.224, 0.225]`.

The first approach is to manually create a transform that fits the given requirements.

In [16]:
manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

The second approach is to automatically create the transform from the model weights we would like to use.

In [18]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [19]:
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=auto_transforms,
    batch_size=32
)